<a href="https://colab.research.google.com/github/prady00/llm-fine-tuning/blob/main/Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install Required Libraries

!pip install -q peft transformers accelerate datasets bitsandbytes huggingface_hub


In [14]:
# Step 2: Read HF token securely from Colab userdata
import os
from google.colab import userdata
from huggingface_hub import HfFolder

hf_token = userdata.get("HF_TOKEN")  # stored via Colab UI
os.environ["HF_TOKEN"] = hf_token
HfFolder.save_token(hf_token)  # optional: saves to ~/.huggingface/token


In [15]:
# Step 3: Import Libraries
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig, get_peft_model, TaskType
)


In [16]:
# Step 4: Load Model & Tokenizer
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model = prepare_model_for_kbit_training(model)

In [17]:
# Step 5: Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [29]:
# Step 6: Load and Format Dataset
dataset = load_dataset("Abirate/english_quotes", split="train")
dataset = dataset.select(range(100))  # keep it light for Colab

def format(example):
    return {
        "text": f"### Instruction: Generate a continuation for this quote:\n{example['quote']}\n### Response:"
    }

dataset = dataset.map(format)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

,quote,author,tags,text
0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"[be-yourself, gilbert-perreira, honesty, inspi...",### Instruction: Generate a continuation for t...
1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"[best, life, love, mistakes, out-of-control, t...",### Instruction: Generate a continuation for t...
2,“Two things are infinite: the universe and hum...,Albert Einstein,"[human-nature, humor, infinity, philosophy, sc...",### Instruction: Generate a continuation for t...
3,"“So many books, so little time.”",Frank Zappa,"[books, humor]",### Instruction: Generate a continuation for t...
4,“A room without books is like a body without a...,Marcus Tullius Cicero,"[books, simile, soul]",### Instruction: Generate a continuation for t...


In [19]:
# Step 7: Tokenize
tokenized_dataset = dataset.map(
    lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=128),
    batched=True
)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [20]:
# Step 8: Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


4.52.4


In [21]:
# Step 9: Training Arguments
repo_name = "tinyllama-lora-english-quotes"

training_args = TrainingArguments(
    output_dir=repo_name,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    # evaluation_strategy="epoch",  # ✅ optional, remove if you don’t want eval
    save_total_limit=1,
    report_to="none",
    push_to_hub=True,
    hub_model_id=repo_name
)

In [22]:
# Step 10: Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-22-2031205137.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Step 11: Train and Push
trainer.train()
trainer.push_to_hub()
tokenizer.push_to_hub(repo_name)

In [33]:
# Step 12: Test Generation
model.eval()
prompt = "### Instruction: Write a motivational quote\n### Response:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction: Write a motivational quote
### Response:

"The greatest glory that a man can find in life is not the victory of his own heart, but the victory of his heart's love for his fellow man."

This quote comes from a poem by poet John Keats, and it's a beautiful expression of the power of love to bring us together and lift us up. It reminds us that even in the darkest of times, there can be light, and that love is the fuel that drives us towards a better future.

I hope this quote inspires you to find love in the world around you, to support and uplift those around you, and to remember the power of the human connection.
